In [1]:
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.model_selection import KFold, StratifiedKFold

from pytorch_tabnet.pretraining import TabNetPretrainer
from pytorch_tabnet.tab_model import TabNetClassifier
import torch
from sklearn.preprocessing import StandardScaler

In [2]:
feat = "feat00"
train = pl.read_csv(f"feat/feat_train_{feat}.csv")
test = pl.read_csv(f"feat/feat_test_{feat}.csv")
train_origin = pl.read_csv("data/train.csv").rename({"": "idx"})

# 説明変数のカラム
cols_exp = [c for c in test.columns if c != "idx"]

# カテゴリ特徴量のカラム
cols_notcat = ['idx', 'created_at', 'tree_dbh']
cols_cat = [c for c in test.columns if not c in cols_notcat] # カテゴリ特徴量

for col in cols_cat:
    # もし欠損値があればその特徴量のユニーク数(欠損値を除く)で埋める（ordinal encoding前提）
    num_null = train[col].n_unique() - 1
    train = train.with_columns(train[col].fill_null(num_null))
    test = test.with_columns(test[col].fill_null(num_null))

# 100以上のユニーク数をもつ特徴量を削除（train/valid split時にordinal encodingの連番の整合性が崩れるため）
# ※ pretrain実施しない場合はコメントアウトする
cols_exp = [c for c in cols_exp if not c in ("boro_ct", "spc_latin", "nta")]
cols_cat = [c for c in cols_cat if not c in ("boro_ct", "spc_latin", "nta")]

### Tabnet

In [3]:
def train_tabnet(train, cols_exp, cols_cat, col_target, params=None):
    if params is None:
        params = {}

    params_add = {"device_name": "cpu", "seed": 0}
    params |= params_add

    x = train[cols_exp].to_numpy()
    y = train[col_target].to_numpy()

    # cols_expにおけるカテゴリ変数のインデックス
    cols_cat_idxs = [i for i, c in enumerate(cols_exp) if c in cols_cat]
    cols_num_idxs = [i for i, c in enumerate(cols_exp) if not c in cols_cat]
    cols_cat_dims = train.approx_n_unique()[cols_exp].to_numpy().ravel()[cols_cat_idxs]

    # K-fold
    kf = KFold(n_splits=5, shuffle=True, random_state=0)
    y_valid_pred_lst = []
    idx_valid_lst = []
    clf_lst = []

    # cross validation
    for fold, (idx_train, idx_valid) in enumerate(kf.split(x)):
        print("fold", fold)
        x_train = x[idx_train, :]
        x_valid = x[idx_valid, :]
        y_train = y[idx_train]
        y_valid = y[idx_valid]
        
        # normalization
        scaler = StandardScaler()
        x_train[:, cols_num_idxs] = scaler.fit_transform(x_train[:, cols_num_idxs])
        x_valid[:, cols_num_idxs] = scaler.transform(x_valid[:, cols_num_idxs])

        # modeling
        pretrainer = TabNetPretrainer(**params)
        pretrainer.fit(x_train, eval_set=[x_valid])
        clf = TabNetClassifier(**params, cat_idxs=cols_cat_idxs, cat_dims=cols_cat_dims)
        clf.fit(
            x_train, y_train,
            eval_set=[(x_train, y_train), (x_valid, y_valid)],
            eval_name=['train', 'valid'],
            eval_metric=["logloss"], 
            from_unsupervised=pretrainer
        )   
        
        # oof
        y_valid_pred = clf.predict_proba(x_valid)
        y_valid_pred_lst.append(y_valid_pred)
        idx_valid_lst.append(idx_valid)
        clf_lst.append(clf)

    idx_valid = np.hstack(idx_valid_lst)
    y_valid_pred = np.vstack(y_valid_pred_lst)
    oof_pred = y_valid_pred[np.argsort(idx_valid)]

    return clf_lst, oof_pred

In [4]:
def predict_test(x_test, clf_lst):
    y_test_pred_lst = []

    for clf in clf_lst:
        y_test_pred = clf.predict_proba(x_test)
        y_test_pred_lst.append(y_test_pred)

    y_test_pred = np.mean(y_test_pred_lst, axis=0)
    return y_test_pred

In [5]:
col_target = "health"

# train tabnet
clf_lst, oof_pred = train_tabnet(train, cols_exp, cols_cat, col_target)

# normalization for test
x_train = train[cols_exp].to_numpy()
x_test = test[cols_exp].to_numpy()
cols_num_idxs = [i for i, c in enumerate(cols_exp) if not c in cols_cat]
scaler = StandardScaler()
scaler.fit(x_train[:, cols_num_idxs])
x_test[:, cols_num_idxs] = scaler.transform(x_test[:, cols_num_idxs])

# predict test with CV ensemble
y_test_pred = predict_test(x_test, clf_lst)

# record
oof_pred_df = pl.DataFrame(oof_pred, schema=[f"health_is_{h}" for h in range(3)])
test_pred_df = pl.DataFrame(y_test_pred, schema=[f"health_is_{h}" for h in range(3)])

fold 0


d:\study\venv\lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 26.76198| val_0_unsup_loss_numpy: 20.42473030090332|  0:00:00s
epoch 1  | loss: 4.08857 | val_0_unsup_loss_numpy: 10.79423999786377|  0:00:01s
epoch 2  | loss: 1.95025 | val_0_unsup_loss_numpy: 5.092899799346924|  0:00:02s
epoch 3  | loss: 1.46901 | val_0_unsup_loss_numpy: 3.2443299293518066|  0:00:03s
epoch 4  | loss: 1.27537 | val_0_unsup_loss_numpy: 1.1792399883270264|  0:00:04s
epoch 5  | loss: 1.1298  | val_0_unsup_loss_numpy: 1.4030499458312988|  0:00:05s
epoch 6  | loss: 1.08237 | val_0_unsup_loss_numpy: 1.2402499914169312|  0:00:06s
epoch 7  | loss: 1.20298 | val_0_unsup_loss_numpy: 1.3411699533462524|  0:00:07s
epoch 8  | loss: 1.13437 | val_0_unsup_loss_numpy: 1.1023600101470947|  0:00:08s
epoch 9  | loss: 1.00452 | val_0_unsup_loss_numpy: 1.2569999694824219|  0:00:09s
epoch 10 | loss: 1.02615 | val_0_unsup_loss_numpy: 1.0763499736785889|  0:00:09s
epoch 11 | loss: 0.99021 | val_0_unsup_loss_numpy: 0.9975200295448303|  0:00:10s
epoch 12 | loss: 1.00271 | val_

d:\study\venv\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
d:\study\venv\lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
d:\study\venv\lib\site-packages\pytorch_tabnet\abstract_model.py:118: UserWarning: Pretraining: cat_dims changed from [2, 4, 4, 2, 3, 5, 45, 59, 26, 65, 51, 2, 2, 2, 2, 2, 2, 2, 2, 2] to []
  warnings.warn(wrn_msg)
d:\study\venv\lib\site-packages\pytorch_tabnet\abstract_model.py:118: UserWarning: Pretraining: cat_emb_dim changed from [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] to []
  warnings.warn(wrn_msg)
d:\study\venv\lib\site-packages\pytorch_tabnet\abstract_model.py:118: UserWarning: Pretraining: cat_idxs changed from [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21] to []
  warnings.warn(wrn_msg)
d:\study\venv\lib\site-packages\pytorch_tabnet

epoch 0  | loss: 0.74412 | train_logloss: 0.66233 | valid_logloss: 0.67185 |  0:00:01s
epoch 1  | loss: 0.62741 | train_logloss: 0.63318 | valid_logloss: 0.64351 |  0:00:02s
epoch 2  | loss: 0.6168  | train_logloss: 0.61387 | valid_logloss: 0.61974 |  0:00:03s
epoch 3  | loss: 0.61381 | train_logloss: 0.61287 | valid_logloss: 0.61755 |  0:00:04s
epoch 4  | loss: 0.61177 | train_logloss: 0.61425 | valid_logloss: 0.62027 |  0:00:05s
epoch 5  | loss: 0.61214 | train_logloss: 0.61341 | valid_logloss: 0.61928 |  0:00:06s
epoch 6  | loss: 0.61252 | train_logloss: 0.61258 | valid_logloss: 0.61788 |  0:00:07s
epoch 7  | loss: 0.61112 | train_logloss: 0.60901 | valid_logloss: 0.6166  |  0:00:08s
epoch 8  | loss: 0.61022 | train_logloss: 0.60804 | valid_logloss: 0.61677 |  0:00:09s
epoch 9  | loss: 0.61043 | train_logloss: 0.60974 | valid_logloss: 0.61802 |  0:00:10s
epoch 10 | loss: 0.61256 | train_logloss: 0.61022 | valid_logloss: 0.61836 |  0:00:12s
epoch 11 | loss: 0.61066 | train_logloss: 0

...

d:\study\venv\lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 28.38081| val_0_unsup_loss_numpy: 116.68115234375|  0:00:00s
epoch 1  | loss: 3.30085 | val_0_unsup_loss_numpy: 6.321189880371094|  0:00:01s
epoch 2  | loss: 1.67137 | val_0_unsup_loss_numpy: 3.409219980239868|  0:00:02s
epoch 3  | loss: 1.36336 | val_0_unsup_loss_numpy: 2.843679904937744|  0:00:03s
epoch 4  | loss: 1.2589  | val_0_unsup_loss_numpy: 2.1197500228881836|  0:00:04s
epoch 5  | loss: 1.27764 | val_0_unsup_loss_numpy: 1.4433799982070923|  0:00:05s
epoch 6  | loss: 1.12332 | val_0_unsup_loss_numpy: 2.4402499198913574|  0:00:06s
epoch 7  | loss: 1.06985 | val_0_unsup_loss_numpy: 1.2499099969863892|  0:00:07s
epoch 8  | loss: 1.15865 | val_0_unsup_loss_numpy: 2.115540027618408|  0:00:08s
epoch 9  | loss: 1.01649 | val_0_unsup_loss_numpy: 1.3405499458312988|  0:00:08s
epoch 10 | loss: 1.03331 | val_0_unsup_loss_numpy: 1.1784499883651733|  0:00:09s
epoch 11 | loss: 1.02818 | val_0_unsup_loss_numpy: 1.2469899654388428|  0:00:10s
epoch 12 | loss: 0.98553 | val_0_un

d:\study\venv\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
d:\study\venv\lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
d:\study\venv\lib\site-packages\pytorch_tabnet\abstract_model.py:118: UserWarning: Pretraining: cat_dims changed from [2, 4, 4, 2, 3, 5, 45, 59, 26, 65, 51, 2, 2, 2, 2, 2, 2, 2, 2, 2] to []
  warnings.warn(wrn_msg)
d:\study\venv\lib\site-packages\pytorch_tabnet\abstract_model.py:118: UserWarning: Pretraining: cat_emb_dim changed from [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] to []
  warnings.warn(wrn_msg)
d:\study\venv\lib\site-packages\pytorch_tabnet\abstract_model.py:118: UserWarning: Pretraining: cat_idxs changed from [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21] to []
  warnings.warn(wrn_msg)
d:\study\venv\lib\site-packages\pytorch_tabnet

epoch 0  | loss: 0.73888 | train_logloss: 0.66243 | valid_logloss: 0.68047 |  0:00:01s
epoch 1  | loss: 0.62156 | train_logloss: 0.62205 | valid_logloss: 0.64172 |  0:00:02s
epoch 2  | loss: 0.61597 | train_logloss: 0.61653 | valid_logloss: 0.63132 |  0:00:03s
epoch 3  | loss: 0.61316 | train_logloss: 0.61374 | valid_logloss: 0.6285  |  0:00:04s
epoch 4  | loss: 0.61226 | train_logloss: 0.61406 | valid_logloss: 0.63053 |  0:00:05s
epoch 5  | loss: 0.612   | train_logloss: 0.612   | valid_logloss: 0.6232  |  0:00:06s
epoch 6  | loss: 0.61168 | train_logloss: 0.61004 | valid_logloss: 0.628   |  0:00:07s
epoch 7  | loss: 0.6106  | train_logloss: 0.60701 | valid_logloss: 0.62141 |  0:00:08s
epoch 8  | loss: 0.60685 | train_logloss: 0.60753 | valid_logloss: 0.62426 |  0:00:09s
epoch 9  | loss: 0.60839 | train_logloss: 0.60518 | valid_logloss: 0.62114 |  0:00:11s
epoch 10 | loss: 0.60828 | train_logloss: 0.60609 | valid_logloss: 0.62484 |  0:00:12s
epoch 11 | loss: 0.60835 | train_logloss: 0

d:\study\venv\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [6]:
# save
oof_pred_df.write_csv(f"pred/oof_pred_tabnet_{feat}.csv")
test_pred_df.write_csv(f"pred/test_pred_tabnet_{feat}.csv")